In [13]:
from googleapiclient.discovery import build
import pandas as pd
import numpy as np
import isodate
from isodate import parse_duration
from IPython.display import JSON 
from datetime import datetime
import mysql.connector
import pymysql
import decimal

In [14]:
# Functions to convert data properly
    # Define a function to convert time strings to seconds
def time_to_seconds(time_str):
    duration = parse_duration(time_str)
    return duration.total_seconds()
    # Define a function to get the day of the week
def day_of_week(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
    day_of_week = date_obj.strftime('%A')
    return day_of_week
    # Define a function to get the date
def date_only(date_str):
    date_obj = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
    date_only = date_obj.date()
    return date_only


In [15]:
# API key created on https://console.cloud.google.com/ under divide music project
api_key = '#####'
channel_id = 'UC3P-tn_Mk1V_KE8M2On8F2w'
playlist_id = 'UU3P-tn_Mk1V_KE8M2On8F2w'
                


In [16]:


api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client for Channel Stats
youtube = build(
    api_service_name, api_version, developerKey=api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=channel_id
)
response = request.execute()

JSON(response)


<IPython.core.display.JSON object>

In [17]:
# Format Channel Stats to a usable table
all_data = []

for item in response['items']:
    data = {'channel_id': item['id']
            , 'channel_name': item['snippet']['title']
            , 'channel_description': item['snippet']['description']
            , 'channel_created_date': item['snippet']['publishedAt']
            , 'channel_subscribers': item['statistics']['subscriberCount']
            , 'channel_views': item['statistics']['viewCount']
            , 'channel_videos': item['statistics']['videoCount']
            , 'channel_playlist_id': item['contentDetails']['relatedPlaylists']['uploads']
    }

    all_data.append(data)

channel_stats = pd.DataFrame(all_data)
# Change the data types of numeric variables
channel_stats['channel_created_date'] = channel_stats['channel_created_date'].apply(date_only)
channel_stats

,channel_id,channel_name,channel_description,channel_created_date,channel_subscribers,channel_views,channel_videos,channel_playlist_id
0,UC3P-tn_Mk1V_KE8M2On8F2w,Divide Music,Official channel of artist Divide Music. All s...,2016-08-20,316000,174170258,174,UU3P-tn_Mk1V_KE8M2On8F2w


In [18]:
# Get credentials and create an API client for the Playlist
# Retrieve the first 50 video IDs, this is the max results allowed
request = youtube.playlistItems().list(
    part="snippet,contentDetails",
    playlistId = playlist_id,
    maxResults = 50
)

response = request.execute()

video_ids = []

for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

# Run a While Loop to retrieve all the video Id's
next_page_token = response.get('nextPageToken')
while next_page_token is not None:
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50,
        pageToken = next_page_token
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId']) 
    
    next_page_token = response.get('nextPageToken')

len(video_ids)

174

In [7]:
# Format Video Stats to a usable table
all_data = []

for i in range(0, len(video_ids), 1):
    request = youtube.videos().list(
    part = "snippet, contentDetails, statistics",
    id = video_ids[i]
    )
    response = request.execute()
    for item in response['items']:
        data = {'video_id': item['id']
                , 'video_name': item['snippet']['title']
                , 'video_created_date': item['snippet']['publishedAt']
                , 'video_likes': item['statistics']['likeCount']
                , 'video_views': item['statistics']['viewCount']
                , 'video_comment': item['statistics']['commentCount']
                , 'video_duration': item['contentDetails']['duration']
        }

    all_data.append(data)

video_stats = pd.DataFrame(all_data)
video_stats.head(10)

,video_id,video_name,video_created_date,video_likes,video_views,video_comment,video_duration
0,hu9e9J1khZA,Head to my IG @divide.music to enter my giveaw...,2023-09-12T03:10:24Z,346,5155,6,PT59S
1,T_AH2u-MUB0,🔥🤘FALL MERCH GIVEAWAY 🤘🔥 Head to my IG @divide...,2023-09-08T22:19:43Z,226,2034,10,PT5S
2,QC1fHr7arjk,Have you listened to my new song? Would love t...,2023-09-06T17:43:39Z,553,4790,58,PT18S
3,E-hx9egmam0,"Asta Song | ""Never Giving Up"" | Divide Music F...",2023-09-01T17:00:08Z,9804,202821,579,PT3M24S
4,z8JTtQWfK3o,9.1.23 #divide...,2023-08-28T21:56:39Z,646,6116,49,PT7S
5,l3_-adOVg_o,Full video is live on my channel! #newmusic #t...,2023-08-22T01:48:24Z,543,5349,9,PT29S
6,4aoqWIxGlxM,No better feeling 😂🤘#dividemusic #theoffice #n...,2023-08-20T01:32:17Z,1296,10790,48,PT11S
7,E-ZsndOCCwA,"Have you listened to my newest song ""Reclaim""?",2023-08-18T20:54:51Z,808,7485,67,PT31S
8,iAaShjSn-Jg,"Batman Song | ""Reclaim"" | Divide Music [DC Com...",2023-08-04T17:00:35Z,11937,386780,654,PT3M36S
9,trBwS_5JSPw,"CELL SONG | ""Perfection"" | Divide Music Ft. Fa...",2023-07-14T17:00:37Z,12712,478648,833,PT4M6S


In [8]:
# Change the data types of numeric variables
video_stats[['video_likes', 'video_views', 'video_comment']] = video_stats[['video_likes', 'video_views', 'video_comment']].apply(pd.to_numeric, errors = 'coerce', axis = 1)

# Create new columns
video_stats['video_duration_seconds'] = video_stats['video_duration'].apply(time_to_seconds)
video_stats['video_created_day'] = video_stats['video_created_date'].apply(day_of_week)
video_stats['video_created_date'] = video_stats['video_created_date'].apply(date_only)
video_stats['channel_id'] = channel_id
video_stats['short'] = np.where(video_stats['video_duration_seconds']<=60, 1, 0)

video_stats['video_likes'] = video_stats['video_likes'].apply(decimal.Decimal)
video_stats['video_views'] = video_stats['video_views'].apply(decimal.Decimal)
video_stats['video_comment'] = video_stats['video_comment'].apply(decimal.Decimal)
video_stats['video_duration_seconds'] = video_stats['video_duration_seconds'].apply(decimal.Decimal)
video_stats['short'] = video_stats['short'].apply(decimal.Decimal)

video_stats.head(10)


,video_id,video_name,video_created_date,video_likes,video_views,video_comment,video_duration,video_duration_seconds,video_created_day,channel_id,short
0,hu9e9J1khZA,Head to my IG @divide.music to enter my giveaw...,2023-09-12,346,5155,6,PT59S,59,Tuesday,UC3P-tn_Mk1V_KE8M2On8F2w,1
1,T_AH2u-MUB0,🔥🤘FALL MERCH GIVEAWAY 🤘🔥 Head to my IG @divide...,2023-09-08,226,2034,10,PT5S,5,Friday,UC3P-tn_Mk1V_KE8M2On8F2w,1
2,QC1fHr7arjk,Have you listened to my new song? Would love t...,2023-09-06,553,4790,58,PT18S,18,Wednesday,UC3P-tn_Mk1V_KE8M2On8F2w,1
3,E-hx9egmam0,"Asta Song | ""Never Giving Up"" | Divide Music F...",2023-09-01,9804,202821,579,PT3M24S,204,Friday,UC3P-tn_Mk1V_KE8M2On8F2w,0
4,z8JTtQWfK3o,9.1.23 #divide...,2023-08-28,646,6116,49,PT7S,7,Monday,UC3P-tn_Mk1V_KE8M2On8F2w,1
5,l3_-adOVg_o,Full video is live on my channel! #newmusic #t...,2023-08-22,543,5349,9,PT29S,29,Tuesday,UC3P-tn_Mk1V_KE8M2On8F2w,1
6,4aoqWIxGlxM,No better feeling 😂🤘#dividemusic #theoffice #n...,2023-08-20,1296,10790,48,PT11S,11,Sunday,UC3P-tn_Mk1V_KE8M2On8F2w,1
7,E-ZsndOCCwA,"Have you listened to my newest song ""Reclaim""?",2023-08-18,808,7485,67,PT31S,31,Friday,UC3P-tn_Mk1V_KE8M2On8F2w,1
8,iAaShjSn-Jg,"Batman Song | ""Reclaim"" | Divide Music [DC Com...",2023-08-04,11937,386780,654,PT3M36S,216,Friday,UC3P-tn_Mk1V_KE8M2On8F2w,0
9,trBwS_5JSPw,"CELL SONG | ""Perfection"" | Divide Music Ft. Fa...",2023-07-14,12712,478648,833,PT4M6S,246,Friday,UC3P-tn_Mk1V_KE8M2On8F2w,0


In [9]:
# Export Data to CSV
video_stats.to_csv(r'divide_music_videos.csv', index=False, header=True)
channel_stats.to_csv(r'divide_music_channel.csv',index=False, header=True)


In [10]:
# Connect to MySQL server
cnx = mysql.connector.connect(
    host = '####'
    , user = '####'
    , password = '####'
    , database = '####'
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

# Query to drop table
drop_table_query = """
    drop table external_data.divide_music_videos
"""
cursor.execute(drop_table_query)

drop_table_query = """
    drop table external_data.divide_music_channel
"""
cursor.execute(drop_table_query)

# Query to create table
create_table_query = """
    create table external_data.divide_music_videos (
        video_id varchar(50)
        , video_name varchar(200)
        , video_created_date date
        , video_likes float
        , video_views float
        , video_comment float
        , video_duration varchar(50)
        , video_duration_seconds float
        , video_created_day varchar(50)
        , channel_id varchar(50)
        , short boolean
    )
    """
cursor.execute(create_table_query)

create_table_query = """
    create table external_data.divide_music_channel (
        channel_id varchar(50)
        , channel_name varchar(50)
        , channel_description varchar(200)
        , channel_created_date date
        , channel_subscribers float
        , channel_views float
        , channel_videos float
        , channel_playlist_id varchar(50)
    )
    """
cursor.execute(create_table_query)


In [11]:
# Insert Data into the table for Videos
for i in range(len(video_stats)):
    sql_videos = "INSERT INTO external_data.divide_music_videos (video_id, video_name, video_created_date, video_likes, video_views, video_comment, video_duration, video_duration_seconds, video_created_day, channel_id, short) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values_videos = (video_stats['video_id'].iloc[i], video_stats['video_name'].iloc[i], video_stats['video_created_date'].iloc[i], video_stats['video_likes'].iloc[i], video_stats['video_views'].iloc[i], video_stats['video_comment'].iloc[i], video_stats['video_duration'].iloc[i], video_stats['video_duration_seconds'].iloc[i], video_stats['video_created_day'].iloc[i], video_stats['channel_id'].iloc[i], video_stats['short'].iloc[i])
        
    # Execute the INSERT statement
    cursor.execute(sql_videos, values_videos)

# Insert Data into the table for Channel
for i in range(len(channel_stats)):
    sql_Channel = "INSERT INTO external_data.divide_music_channel (channel_id, channel_name, channel_description, channel_created_date, channel_subscribers, channel_views, channel_videos, channel_playlist_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values_Channel = (channel_stats['channel_id'].iloc[i], channel_stats['channel_name'].iloc[i], channel_stats['channel_description'].iloc[i], channel_stats['channel_created_date'].iloc[i], channel_stats['channel_subscribers'].iloc[i], channel_stats['channel_views'].iloc[i], channel_stats['channel_videos'].iloc[i], channel_stats['channel_playlist_id'].iloc[i])
        
    # Execute the INSERT statement
    cursor.execute(sql_Channel, values_Channel)


# Close the connection
cnx.commit()
cnx.close()